# Working in Pixel Coordinates

### import stuff

In [ ]:
import mercantile
import requests
import mapbox_vector_tile
import geopandas as gpd
import shapely.geometry
import matplotlib.pyplot as plt

### define the coordinates and calculate the tile coordinates

$x_i,y_i,z_i$ are the Tile Coordinates of the inner Tile (which will be the Image Tile)  
$x_o,y_o,z_o$ are the Tile Coordinates of the outer Tile

In [ ]:
# Test Coordinates:
# Dinslaken Rotbachsee
lat,lon = 51.56792718343954, 6.78504175412729
# Dortmund Phönixsee
#lat, lon = 51.49011058141199, 7.506262816458572
# Tenderingssee
# lat,lon = 51.59932699198615, 6.719474067819083
z_i = 15
z_o = 10

In [ ]:
tile_i = mercantile.tile(lon,lat,z_i)

In [ ]:
x_i,y_i = tile_i.x,tile_i.y
z_i,x_i,y_i

In [ ]:
tile_o = mercantile.tile(lon,lat,z_o)

In [ ]:
x_o,y_o = tile_o.x,tile_o.y
z_o,x_o,y_o

### Download and read in the Vector Tile of the outer Tile

In [ ]:
tileset_id = 'mapbox.mapbox-streets-v8'

In [ ]:
token = 'pk.eyJ1Ijoibmljb2pnIiwiYSI6ImNrcHFqamY5azNqaTAycHJpa210ZDF2aG4ifQ.8jPQpYl4eiUw0Cv_b8J7hA'

In [ ]:
file_path_mvt = f'data/vector_tiles/z{z_o}x{x_o}y{y_o}_{tileset_id}.mvt'
file_path_mvt

In [ ]:
# download the vector tile if it's not already there
import os.path
if not os.path.isfile(file_path_mvt):
    url = f'https://api.mapbox.com/v4/{tileset_id}/{z_o}/{x_o}/{y_o}.mvt?access_token={token}'
    print(url)
    #r = requests.get(url)  
    #with open(file_path_mvt, 'wb') as f:
    #    f.write(r.content)
else:
    print(f'MVT file already exists')

In [ ]:
# read the vector tile
with open(file_path_mvt, mode='rb') as file:
    vector_tile = mapbox_vector_tile.decode(file.read(),y_coord_down=True)

In [ ]:
vector_tile.keys()

In [ ]:
water_df = gpd.GeoDataFrame.from_features(vector_tile['water']['features'])
water_df

### calculate the bounding box of the inner tile in vector tile pixel coordinates

https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames  
https://www.maptiler.com/google-maps-coordinates-tile-bounds-projection/  

In [ ]:
# pixel width and height of the outer vector tile
extent_o = 4096

In [ ]:
# pixel width and height of one inner tile
extent_i = extent_o/(2**(z_i-z_o))

In [ ]:
# top left Tile in the outer Tile but with the inner zoom
# https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Subtiles
x_o_with_z_i = 2**(z_i-z_o) * x_o
y_o_with_z_i = 2**(z_i-z_o) * y_o

In [ ]:
# pixel coordinates of the top left corner of the inner tile in the vector tile
px_tl = extent_i * (x_i - x_o_with_z_i)
py_tl = extent_i * (y_i - y_o_with_z_i)

In [ ]:
# bottom right corner of the inner tile in vector tile pixel coordinates
px_br = px_tl + extent_i
py_br = py_tl + extent_i

In [ ]:
bbox = shapely.geometry.box(px_tl, py_tl, px_br, py_br)

In [ ]:
print(bbox)

In [ ]:
ax = water_df.plot()
gpd.GeoSeries(bbox).plot(ax=ax,color='red')
plt.gca().invert_yaxis()

In [ ]:
water_df.plot()
plt.xlim(px_tl,px_br)
plt.ylim(py_tl,py_br)
plt.gca().invert_yaxis()

### check if there is water in the bounding box

In [ ]:
# Geometries overlaps if they have more than one but not all points in common, 
# have the same dimension, 
# and the intersection of the interiors of the geometries has the same dimension as the geometries themselves.
water_df.overlaps(bbox)